# Deel 1: preprocessing
De data die we gaan gebruiken is (weer) de wiki-pagina over kanker. De tekst hiervan kun je hier downloaden. Gebruik van alle zinnen die je hierin vindt alleen de zinnen die meer dan tien woorden bevatten. Voeg deze zinnen samen in een lijst data. Als het goed is, is len(data)=59.

In [98]:
from nltk.corpus import stopwords
from numpy.ma.core import append

with(open("wiki.txt", "r")) as file:
    wiki_text = "".join(line.strip() for line in file)

wiki_text = wiki_text.split(".")
wiki_text = [sentence + "." for sentence in wiki_text if len(set(sentence.split())) >= 10]


print(f"wiki_text len: {len(wiki_text)}")
    

wiki_text len: 134


Maak vervolgens een functie preprocess_sentence die een string meekrijgt en die die string opgeschoond teruggeeft. Het opschonen van de string bestaat uit de volgende twee stappen:

+ verwijderen van (in ieder geval) de volgende karakters: /, ., ., ', ,, ", :, ;, (, ) (misschien dat er nog andere karakters zijn die je uit de zinnen wilt halen).
+ verwijderen van de stopwoorden uit de zin; dit zijn woorden die wel nodig zijn voor de grammatica, maar niet per se om de context te bepalen. Je kunt de stopwoorden hier downloaden.

In [99]:
with(open("stopwoorden.txt", "r")) as file:
    stopwoorden = [line.strip() for line in file if line]
    
def preprocess_sentence(sentence):
    sentence = " ".join([word for word in sentence.split() if word not in stopwoorden])
    sentence = "".join([letter for letter in sentence if letter not in "/..\',\":;()"])
    return sentence


Roep nu de functie preprocess_data aan met alle zinnen uit data. Sla het resultaat op in een nieuwe variabele (bijvoorbeeld sentences of corpus). Deze variabele is je corpus.

In [100]:
corpus  = [preprocess_sentence(sentence) for sentence in wiki_text]

Een tweede stap die we moeten zetten is het samenstellen van het vocabulaire: de woorden waaruit onze zinnen zijn samengesteld (net zoals het Nederlandse woordenboek alle woorden bevat waaruit alle Nederlandse zinnen zijn samengesteld). Sla dit op in een tweede variabele (standaard heet dat ding vocab). Als het goed is, zijn er 717 woorden in je vocabulaire.

In [101]:
vocab = list(set("".join(corpus).split()))
print(f"vocab len: {len(vocab)}")
print(vocab)

vocab len: 874
['ingevoegd', 'lymfotroop', 'is', 'stollingsafwijkingenhuidafwijkingenkoorts', 'celdelingIndien', 'bedrijf', 'zorg', 'Genentech', 'directe', 'Kanker', 'benaderingen', 'markt', 'Celsus[1]BetekenisKanker', 'pylori', 'carcinogenenDe', 'kankervormen', 'neoplasie', 'beschadigde', 'bronchuscarcinoomaromatische', 'maagkankerhet', 'signalenBeide', 'celdeling', 'twee', 'sunitinib', 'verwijderen', 'omgezet', 'vernietigen', 'zonlicht', 'succesvolle', 'celdifferentiatieDeze', 'omliggend', 'teveel', 'regelengenen', 'onttrokken', 'gebied', 'röntgenfoto’s', 'niet-ioniserend', 'optreden', 'oorzakelijk', 'zon', 'IARCNational', 'leeftOok', 'kankerVrouwen', 'besmet', 'verlammingen', 'atoombom', 'muurkanker', 'metastaseren', 'angstige', 'bovengenoemde', 'preventief', 'moeten', 'activerenHet', 'remmende', 'genereren', 'spierafwijkingenbloed-', 'belangrijk', 'orgaan', 'geneesmiddelen', 'defecten', 'gevorderde', 'Enkele', 'stimuleren', 'and', 'combinatie', 'verschillende', 'intracellulaire', '

# Stap 2: CBOW paren aanmaken
Maak nu een functie create_pairs die alle zinnen uit het corpus meekrijgt (corpus), en een parameter (w_size) die aangeeft hoe groot het window moet zijn waarmee het algoritme door het corpus loopt (2, in het voorbeeld hierboven). Deze functie loopt per zin met stappen van w_size over de woorden w van de zin heen. Elke iteratie worden de w_size aan de linkerkant én aan de rechterkant van het woord w als context (als input als het ware) gezien en het woord w zelf als output (zie eventueel de beschrijving hierboven). Sla uiteindelijk alle contexten op in een matrix X en alle woorden w in een vector y. Retourneer X en y.

In [102]:
def create_pairs(corpus, w_size):
    X = []
    y = []
    for sentence in corpus: # per zin
        sentence = sentence.split()
        for i in range(w_size, len(sentence) - w_size - 1): # per woord
            contex = []
            for j in range(w_size * 2 +1): # context index
                if j -w_size != 0: # niet y woord
                    contex.append(sentence[i + j -w_size])
            
            X.append([contex])
            y.append([sentence[i]])
    return X, y
            
x , y = create_pairs(corpus, 2)

print(corpus[0])
print(x[0])
print(y[0])
        

Kanker medisch Latijn neoplasma malignum kwaadaardig nieuwgevormd weefsel ziekte uiter cellen onbeheerst vermenigvuldigen doende woekerende cellen omliggend weefsel richten schade invasieve groei infiltratiede woekerende cellen verspreiden plaatsen lichaam metastasering uitzaaiing lymfevaten lymfogene metastasering bloed hematogene metastasering verschillende lichaamsholten bijvoorbeeld buikholte
[['Kanker', 'medisch', 'neoplasma', 'malignum']]
['Latijn']
